In [36]:
"""
desired outputs:
    -path to json log
    -project type
    -report name
    -run start date/time
    -yes/no if report had error
    -if yes error, error message
    -anything else that gives insight into root cause
    
Really need would be summary of total runs, % failed, breakdown of failure causes.
"""

from pathlib import Path
import json

import pandas as pd

def parse_json(json_in, run_id=0):
    with open(json_in, 'r') as f:
        jloaded = json.load(f)
        
    out_rows = []
    
    base_row = {}
    base_row['run_id'] = run_id
    base_row['log_path'] = str(json_in)
    base_row['start_time'] = jloaded['start']
    base_row['projectType'] = 'unknown'
    base_row['projectName'] = 'unknown'
    base_row['rpt_name'] = 'unknown'
    base_row['email'] = 'unknown'
    base_row['error_flag'] = 1
    base_row['error_msg'] = 'Other error. See JSON for details.'
    
    try:
        jbase = jloaded['artifacts'][0]['outputs']['output']
        base_row['projectType'] = jbase['projectType']
        base_row['projectName'] = jbase['projectName']
        base_row['email'] = jbase['email']
        
        for rpt in jbase['reports']:
            rpt_row = base_row
            rpt_row['rpt_name'] = rpt['name']

            rpt_error = False
            rpt_errmsg = ''
            messages = rpt['gpJob']['messages']
            if len(messages) > 0:
                for msg in messages:
                    if msg['type'] == 'esriJobMessageTypeError':
                        rpt_error = True
                    if rpt_error and msg['type'] == 'esriJobMessageTypeInformative':
                        rpt_errmsg = msg['description']

            rpt_row['error_flag'] = int(rpt_error)
            rpt_row['error_msg'] = rpt_errmsg

            out_rows.append(rpt_row)
    except KeyError: # if key error, means significant error and JSON log did not generate as normal
        out_rows.append(base_row)

        
            
    return out_rows

print("modules loaded.")



modules loaded.


In [39]:
# NOTE - to run this script, must be logged in to admin or other account authorized to read from the VGIS workflow folder
root_dir = Path(r'\\WEBMAPPING-SVR\Workflow')
subdirs = ['0', '1', '2', '3', '4', '5', '6', '7', '8'] # subfolder names within Workflow folder

null_row = {'run_id': run_n,
            'log_path': 'na',
            'start_time': 'na',
            'projectType': 'unknown',
            'projectName': 'unknown',
            'rpt_name': 'unknown',
            'email': 'unknown',
            'error_flag': 1,
            'error_msg': 'Other error. See JSON for details.'
           }

data_rows = []
run_n = 0
for sd in subdirs:
    dpth = root_dir.joinpath(sd)
    logpaths = [i.joinpath('Logs') for i in dpth.glob('*') if i.is_dir()]
    for lp in logpaths:
        log_files = [i for i in lp.glob('*.json')]
        if len(log_files) > 1:
            null_row['log_path'] = str(lp)
            data_rows.append(null_row)
        else:
            log_data = parse_json(log_files[0], run_id=run_n)
            data_rows += log_data
            run_n += 1


df = pd.DataFrame(data_rows)
unique_logs = df.run_id.drop_duplicates().shape[0]
print(f"{unique_logs} run logs summarized")




1008 run logs summarized


In [35]:
df.shape


(4824, 9)

error_flag
0    878
1    130
Name: count, dtype: int64

In [14]:
jbase['reports'][0]['gpJob']['messages']

[]